In [1]:
import os
import ast
import time
from tqdm import tqdm
import json
from glob import glob
from collections import Counter, defaultdict
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np

from load_data import load_user_time_read
from tfidf import load_tfidf
from config import DataRoots
from utils import iterate_data_files, squeeze
from preprocessing import PostIdEncoder

In [26]:
from load_data import load_post_id_encoder

In [3]:
start = time.time()

user_time_read = load_user_time_read(root_dir='../preprocessed/user_time_read.json')

print(f'Loading time: {time.time() - start}')

Loading time: -15.045081853866577


In [17]:
user_list = list(user_time_read.keys())

In [22]:
history = user_time_read[user_list[0]] # 한 유저의 히스토리

In [24]:
history

[['2018100100_2018100101', ['@kty0613_91']],
 ['2018100200_2018100201', ['@svillustrated_43', '@svillustrated_44']],
 ['2018100318_2018100319', ['@jericho27_410', '@enamoramiento87_277']],
 ['2018100509_2018100510', ['@dapls_238', '@dapls_249']],
 ['2018100600_2018100601', ['@clncompany_168']],
 ['2018100722_2018100723',
  ['@ida0724_22',
   '@ida0724_11',
   '@alexkang_905',
   '@alexkang_900',
   '@yunta2_844']],
 ['2018100818_2018100819', ['@finance1026_211']],
 ['2018100918_2018100919', ['@alexkang_910']],
 ['2018101002_2018101003', ['@peregrino97_826']],
 ['2018101014_2018101015', ['@mobiinside_1281']],
 ['2018101117_2018101118',
  ['@alexkang_913',
   '@roysday_262',
   '@jordan777_2328',
   '@jordan777_2328',
   '@jordan777_2333']],
 ['2018101118_2018101119', ['@fishz_354', '@wndud961_49']],
 ['2018101121_2018101122', ['@newslaw_14', '@ysp0722_1128']],
 ['2018101205_2018101206', ['@ge999_14', '@chacha0401_45', '@flowerdog_370']],
 ['2018101206_2018101207', ['@kihyonkim_10']],
 [

In [12]:
path = {'train': '../preprocessed/train', 'dev': '../preprocessed/dev'}

In [14]:
dev = open(path['dev'], 'r').readlines()

In [16]:
dev

'#a0923400c3255b40221fcf35a0735bd4 @brunch_151 @syshine7_56 @cli-annah_639 @taekangk_61 @2highlight_63 @brunch_151\n'

In [6]:
len(user_time_read)

306222

재구성된 read 데이터
```python
{
    '#유저식별자1': 
        {
            '날짜1': [조회글1, 조회글2, ...],
            '날짜2': [조회글1, 조회글2, ...], 
            ...
        },
    '#유저식별자2':
        {
            '날짜1': [조회글1, 조회글2, ...],
            '날짜2': [조회글1, 조회글2, ...], 
            ...
        }
    
}
```

In [4]:
reads.head(1000)

NameError: name 'reads' is not defined

In [127]:
reads_subset = reads.head(1000)

In [128]:
user_private_list = reads_subset['user_private'].unique().tolist()

In [103]:
sample = user_private_list[i]
reads_user = reads.loc[reads['user_private']==sample]
reads_user = reads_user.drop('user_private', axis=1)

In [ ]:
defaultdict

In [110]:
user_log_dict = defaultdict(list)
for _, series in reads_user.iterrows():
    log_piece = {series['start_time']: series['sequence']}
    user_log_dict[sample].append(log_piece)

In [93]:
time_logs = reads_user['start_time'].tolist() # start time of log

In [98]:
reads_user.iloc[0, :]

start_time        2019012203
sequence      [@dong02_1342]
Name: 39, dtype: object

In [82]:
reads_user.shape

(88, 3)

88

In [2]:
tfidf_dir = Config.tfidf_root
vocab_dir = Config.vocab_root
data_root = Config.data_root

In [3]:
load_tfidf(post_meta_id_list=[1,2,3], tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=False)

,post_meta_id,여행,영화,사랑,에세이,그림일기,생각,공감에세이,일상,글쓰기,감성에세이,인생,행복,연애,소설,드로잉,독서,육아,그림,마음,사진,리뷰,엄마,음악,스타트업,시,유럽여행,일러스트,이별,마케팅,고양이,디자인,영화리뷰,웹툰,IT,직장인,감성,일기,사람,결혼,...,식도락,서민,F1,라디오헤드,연민,운동방법,여교사,하체,한약,족저근막염,좋은음악,총기,폭탄,도파민,산행후기,간병인,시화,고요함,멤버십,푸조,유럽연합,매체,눈높이,댄서,대상포진,블랙박스,불가능,연수,외침,신트라,삿포로여행,김수현,고부갈등,배달음식,의류,본인,체험학습,선물투자,시금치,벌금
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.268116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
START_DEV = '2018100100'
END_DEV = '2018101000'
ROOT = '../raw'

In [7]:
post_meta = pd.read_csv(os.path.join(ROOT, '../raw/preprocessed/post_meta_id.csv'))

post_meta_id = post_meta['post_meta_id'].tolist()
post_id = post_meta['post_id'].tolist()

post_id_encoding = {i:j for i, j in zip(post_id, post_meta_id)}
post_id_decoding = {i:j for i, j in zip(post_meta_id, post_id)}

In [5]:
records = [[line.strip().split() for line in open(path)] for path, _ in tqdm(iterate_data_files(from_dtm=START_DEV, to_dtm=END_DEV, root_dir=ROOT))]r

216it [00:44,  4.81it/s]


In [22]:
path = glob('../raw/read/*')[0]

In [57]:
read_list = glob('../raw/read/*')[:1500]

In [59]:
start = time.time()

lines = []
for path in read_list:
    lines.append([line.split() for line in open(path)])

print((time.time() - start) / 60)

0.11149210532506307


In [ ]:
def load_read(path: str) -> list:
    return [line.split() for line in open(path)]

In [58]:
start = time.time()

output = pd.Series(read_list).apply(lambda x: load_read(x))

print((time.time() - start) / 60)

3.862061850229899


In [21]:
pd.read_csv(read_list[0])

,#ac9d9762cb49332f6e2cd153ba91434c @dalda_256 @dalda_256 @seongheeleelrwn_2 @elliyalove_128 @thecapitalist_178 @thecapitalist_178 @jigunamja_1571 @tjkmix_416 @yurigin_115 @yurigin_115 @yurigin_114
0,#b6a40aa32c58fe28e2b823189bffddd0 @dailylife_2...
1,#c85abb6187a05fe5f37eabae6e6e5c0d @hedijung_93
2,#7b321056618b7dfe6723b7db437f4f4e @bang1999_46...
3,#a8a993fd98c4b2ede6296474e6aec399 @hitchwill_1...
4,#a29f5a6f6a6dcfb245ac945c2836a085 @lynnata_191...
...,...
333,#5a4a085241e6c227238e3c0a1fdd8d24 @hitchwill_1...
334,#59f0bdda5c68a2d5c341add4d4a98cde @beautyinsig...
335,#6116ce0d156cd8a3df7e11fd7a29897e @designforhu...
336,#b5ddf18495eaff52385296b65671603d @paradic_220...


In [104]:
for record in tqdm(records[:5]):
    for r in record:
        user, posts_raw = r[0], r[1:]
        posts = [post_id_dict[p] for p in posts_raw if p in post_id]
        user_logs[user].extend(posts)




  0%|          | 0/5 [00:00<?, ?it/s]


 20%|██        | 1/5 [00:28<01:55, 28.85s/it]


 40%|████      | 2/5 [01:28<01:53, 37.98s/it]


 60%|██████    | 3/5 [01:53<01:08, 34.28s/it]


 80%|████████  | 4/5 [02:38<00:37, 37.55s/it]


100%|██████████| 5/5 [03:19<00:00, 39.94s/it]


In [111]:
sample_id_list = user_logs['#fb3b42b92d96f00afd5bf62d3575db53']

In [112]:
tfidf = load_tfidf(post_meta_id_list=sample_id_list, tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=True)

In [113]:
tfidf = tfidf.apply(lambda x: x / np.linalg.norm(x), axis=1)

In [114]:
user_preference = tfidf.sum(axis=0)

In [121]:
user_preference = user_preference / np.linalg.norm(user_preference)

In [122]:
user_preference

여행      0.000000
영화      0.000000
사랑      0.186879
에세이     0.000000
그림일기    0.217054
          ...   
본인      0.000000
체험학습    0.000000
선물투자    0.000000
시금치     0.000000
벌금      0.000000
Length: 7000, dtype: float64

In [135]:
post_subsample = np.random.choice(600000, 1000, replace=False).tolist() # 가중 확률도 괜찮겠다

In [137]:
tfidf_subsample = load_tfidf(post_meta_id_list=post_subsample, tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=True)

array([0.        , 0.        , 0.18687906, ..., 0.        , 0.        ,
       0.        ])

In [151]:
recommend_list = (tfidf_subsample * user_preference.values).sum(axis=1).sort_values(ascending=False).head(100).index.tolist()

In [154]:
recommend_list_decoded = [post_id_decode[i] for i in recommend_list]

In [155]:
recommend_list_decoded

['@tallguy_213',
 '@sharppen01_20',
 '@moonwindtre_240',
 '@bookfit_3642',
 '@haninorway19_103',
 '@ameliekim2016_38',
 '@chkwon91_10',
 '@mobiinside_559',
 '@leedongdong_58',
 '@psychorops_48',
 '@slist_86',
 '@drawingbutler_10',
 '@iampact_27',
 '@begini_379',
 '@ranrandambee_53',
 '@triple_26',
 '@littlechamber_16',
 '@ddaddassi_112',
 '@peregrino97_872',
 '@ubermensch_29',
 '@theedit_366',
 '@capotea_118',
 '@tomssi_55',
 '@antifreezie_2',
 '@yooym2000_162',
 '@byulkim_102',
 '@mori3_52',
 '@tiara8743_31',
 '@kateyoungheekwo_26',
 '@byulkim_106',
 '@eunmikim_146',
 '@jamon_2',
 '@hotelscomkr_45',
 '@9320km_50',
 '@csco2pw2gt_27',
 '@simplestories_63',
 '@bonfire_331',
 '@gunbbang_408',
 '@sound-travel_42',
 '@elang8151_222',
 '@simplestories_65',
 '@kohwang56_87',
 '@vhfhflya_45',
 '@glassdice_22',
 '@futurewave_492',
 '@garbar_384',
 '@lasw2324_28',
 '@eewol_41',
 '@gipyung_159',
 '@doyeonsunim_168',
 '@brunchfkx_8',
 '@marupress_149',
 '@byulkim_105',
 '@rana-na_26',
 '@kimssiban